In [1]:
%run homogeneousRelation.ipynb

R = {(1, 1),(1, 2),(2, 1),(3, 3)}
S = {(1, 3),(2, 2),(3, 1)}
R∘S = {(1, 2),(1, 3),(2, 3),(3, 1)}

is binary relation? R = True
is binary relation? S = True

is homogeneous relation? R = True
is homogeneous relation? S = True

is identity relation? R = False
is identity relation? S = False

S[A] = FiniteSet(2, 3)
S-1[A] = FiniteSet(2, 3)

dom S = FiniteSet(1, 2, 3)
rng S = FiniteSet(1, 2, 3)

R == S: False
R != S: True
R < S: False
R <= S: False
R > S: False
R >= S: False

R union S = {(1, 1),(1, 2),(1, 3),(2, 1),(2, 2),(3, 1),(3, 3)}
R intersection S = {}
R difference S = {(1, 1),(1, 2),(2, 1),(3, 3)}

R complement = {(1, 3),(2, 2),(2, 3),(3, 1),(3, 2)}
R∘S converse = {(1, 3),(2, 1),(3, 1),(3, 2)}

'1' is in relation with: FiniteSet(1, 2) in R
<class 'sympy.core.numbers.One'>
<class 'sympy.core.numbers.Integer'>
<class 'sympy.core.numbers.Integer'>
True
True
False
False
True


In [2]:
class PartialOrder(HomogeneousRelation):
    
    def __init__(self,relation=None,domain=None):
        super().__init__(relation, domain)

In [4]:
r = PartialOrder({(1,1),(2,2),(3,3)},{1,2,3})
print(f"R = {r}")

R = {(1, 1),(2, 2),(3, 3)}
